# use the output taxonomy tree to manually assign poi category

In [184]:
# parse FourSquare Taxonomy
from src.constants import fn_frsq_taxonomy_json, fn_frsq_taxonomy_csv, fn_frsq_taxonomy_tree, fn_mapping_for_fs
data_dir = 'data/'
frsq_taxonomy_json_path = data_dir + fn_frsq_taxonomy_json
frsq_taxonomy_csv_path = data_dir + fn_frsq_taxonomy_csv
frsq_taxonomy_tree_path = data_dir + fn_frsq_taxonomy_tree
parse_frsq_taxonomy(frsq_taxonomy_json_path, frsq_taxonomy_csv_path, frsq_taxonomy_tree_path, max_level=6)

TypeError: parse_frsq_taxonomy() got an unexpected keyword argument 'max_level'

# check missing, output "to_be_done" file with "unmapped" category

In [1]:
path_existing_mapping = 'data/manual/poi_mapping_for_fs_v1.txt'
from src.ftr_poi import mapping_for_fs

In [35]:
existing_mapping = mapping_for_fs(path_existing_mapping, False)

In [36]:
tree = mapping_for_fs(frsq_taxonomy_tree_path, False)

In [37]:
print tree.shape
tree.head()

(886, 4)


,lv,tag,parents,top_parent
0,2,Arts & Entertainment,/,
1,3,Amphitheater,//Arts & Entertainment,
2,3,Aquarium,//Arts & Entertainment,
3,3,Arcade,//Arts & Entertainment,
4,3,Art Gallery,//Arts & Entertainment,


In [45]:
print existing_mapping.shape
existing_mapping.head()

(521, 4)


,lv,tag,parents,top_parent
0,1,cycling facilities,,cycling facilities
1,2,Bike Rental / Bike Share,/cycling facilities,cycling facilities
2,1,transportation,,transportation
3,2,Bus Station,/transportation,transportation
4,3,Bus Line,/transportation/Bus Station,transportation


In [99]:
merged = tree.merge(existing_mapping, left_on='tag', right_on='tag', how='left')

In [100]:
merged['top_parent'] = merged.top_parent_y.fillna('unmapped')

In [166]:
def get_path(row):
    path = row.parents_x.split('/')[1:]
    path[0] = row.top_parent
    path.append(row.tag)
    return path

def add_path(d, path):
    if len(path)>1:
        p0 = path[0]
        if not p0 in d:
            d[p0] = {}
        add_path(d[p0], path[1:])
    else:
        d[path[0]] = {}

def depth_first(name, d, level=-1):
    if not d:
        return [level*'\t'+name]
    rows = [level*'\t'+name] if level!=-1 else []
    for sub_name, sub_d in d.items():
        rows.extend(depth_first(sub_name, sub_d, level+1))
    return rows

In [170]:
merged['path'] = merged.apply(get_path, axis=1)

In [172]:
with open('data/manual/frsq_poi_mapping_to_be_done.csv', 'w') as f:
    for c, grp in merged.groupby('top_parent'):
        if c!='unmapped':
            f.write(c+'\n')
            for t in grp.tag.values:
                f.write('\t'+t+'\n')
        else:
            unmapped_paths = {}
            for a in grp.path.values:
                add_path(unmapped_paths, a)
            rows = depth_first('', unmapped_paths)
            for r in rows:
                f.write(r+'\n')

# copy to be done to xmind, then classify unmapped frsq categories

In [159]:
to_be_done = mapping_for_fs('data/manual/frsq_poi_mapping_to_be_done.csv', False)

a = to_be_done[['tag', 'top_parent']].merge(merged[['tag', 'top_parent']], left_on='tag', right_on='tag')

(a.top_parent_x==a.top_parent_y).value_counts()

# check duplicates after manually processing unmapped

In [185]:
new_mapping = mapping_for_fs(data_dir+fn_mapping_for_fs,False)

In [186]:
print new_mapping.tag.value_counts().shape
new_mapping.tag.value_counts()

(897L,)


Meyhane                      1
Botanical Garden             1
Town                         1
Candy Store                  1
Pier                         1
Non-Profit                   1
Luggage Store                1
Train Station                1
Betawinese Restaurant        1
Scenic Lookout               1
Print Shop                   1
Hyderabadi Restaurant        1
Outlet Store                 1
Indonesian Meatball Place    1
Basketball Stadium           1
Clothing Store               1
Warehouse                    1
Romanian Restaurant          1
Roller Rink                  1
nightlife spot               1
Platform                     1
Butcher                      1
Museum                       1
College Tennis Court         1
Karnataka Restaurant         1
Creperie                     1
Seafood Restaurant           1
Bagel Shop                   1
Chiropractor                 1
Border Crossing              1
                            ..
Lounge                       1
Skating 